
# 第二部分：Milvus 基础操作 - 使用 Python SDK  [![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/richzw/milvus-workshop/blob/main/ch2/ch2_1.ipynb)
欢迎来到 Milvus Workshop 的第二部分！在这一部分，我们将学习如何使用 Python SDK (PyMilvus) 与 Milvus进行交互，重点关注 Collection 的管理。
**在开始之前，请确保您已经：**
1. 安装了 Milvus 服务并已启动。
2. 安装了 `pymilvus` Python SDK (`pip install pymilvus`).

In [ ]:
!pip install pymilvus==2.5.8

## 2.1 连接 Milvus 并管理 Collections

首先，我们需要连接到正在运行的 Milvus 服务。

In [1]:
# 导入必要的库
from pymilvus import MilvusClient, DataType, FieldSchema, CollectionSchema 

In [2]:
# 连接到 Milvus 服务
# MilvusClient uses a URI or host/port for connection.
MILVUS_HOST = "localhost" # 或者你的 Milvus 服务器 IP
MILVUS_PORT = "19530"
MILVUS_URI = f"http://{MILVUS_HOST}:{MILVUS_PORT}" # Recommended format

# 或者，如果你有用户名和密码 (for Zilliz Cloud or Milvus with auth)
# MILVUS_USER = "username"
# MILVUS_PASSWORD = "password"
# client = MilvusClient(uri=MILVUS_URI, user=MILVUS_USER, password=MILVUS_PASSWORD)

try:
    # 创建 MilvusClient 实例
    client = MilvusClient(
        uri=MILVUS_URI
        # token="YOUR_API_KEY_OR_TOKEN" # For Zilliz Cloud serverless or other token-based auth
        # db_name="default" # Specify database if not default (Milvus 2.2.9+)
    )
    print(f"成功创建 MilvusClient 并连接到 Milvus 服务: {MILVUS_URI}")
    print(f"Milvus server version (via client): {client.get_server_version()}")
except Exception as e:
    print(f"创建 MilvusClient 或连接 Milvus 服务失败: {e}")

成功创建 MilvusClient 并连接到 Milvus 服务: http://localhost:19530
Milvus server version (via client): 2.5.11


### 概念：Collection (集合)

在 Milvus 中，**Collection** 是一组实体的集合，类似于关系型数据库(如MySQL)中的“表”或 Elasticsearch 中的“索引”。

- 一个 Collection 包含一组具有相同结构（Schema）的实体（Entities）。
- 每个实体可以包含多个字段（Fields），其中包括向量字段和标量字段。
- Collection 是进行向量搜索、插入和管理数据的基本单位。

### 实操：定义 Collection Schema (结构)
要创建一个 Collection，首先需要定义它的 **Schema**。Schema 定义了 Collection 中每个字段的名称、数据类型和其他属性。
#### Field 的概念
- **主键字段 (Primary Key Field)**:
  - 用于唯一标识 Collection 中的每一个实体。
  - 每个 Collection 必须有且仅有一个主键字段。
  - 数据类型通常是 `INT64` 或 `VARCHAR`。
  - 在 `FieldSchema` 中通过 `is_primary=True` 指定。
- **向量字段 (Vector Field)**:
  - 用于存储向量嵌入 (embeddings)。这是 Milvus 进行相似性搜索的核心。
  - 数据类型通常是 `FLOAT_VECTOR` (浮点型向量) 或 `BINARY_VECTOR` (二值型向量)。
  - 创建时必须指定向量的维度 `dim`。
- **标量字段 (Scalar Field)**:
  - 用于存储非向量类型的属性数据，如 ID、名称、类别、时间戳等。
  - 可以用于过滤查询结果 (Attribute Filtering) 或作为元数据返回。
  - 支持多种数据类型。

#### 数据类型 (Data Types)
Milvus 支持以下主要的数据类型：
- `DataType.BOOL`: 布尔型 (True/False)
- `DataType.INT8`, `DataType.INT16`, `DataType.INT32`, `DataType.INT64`: 不同位数的整型
- `DataType.FLOAT`, `DataType.DOUBLE`: 单精度和双精度浮点型
- `DataType.STRING`: 字符串类型（通常用于较长文本，UTF-8编码，长度理论上无限制，但实际受限于gRPC传输大小）
- `DataType.VARCHAR`: 变长字符串类型（UTF-8编码，创建时需要指定 `max_length`，最大长度65535）
- `DataType.ARRAY`: 数组类型，可以包含标量数据。需要指定 `element_type` 和 `max_capacity`。
- `DataType.JSON`: JSON 类型，用于存储半结构化数据。
- `DataType.FLOAT_VECTOR`: 浮点型向量。
- `DataType.BINARY_VECTOR`: 二值型向量。

#### 如何定义主键 (Primary Key)
主键字段用于唯一标识每个实体。
- `is_primary=True`
- 数据类型通常为 `INT64` 或 `VARCHAR` (需要指定 `max_length`)。
- 如果主键字段是 `INT64` 类型，可以设置 `auto_id=True` 来让 Milvus 自动生成唯一的 ID。如果为 `VARCHAR`，则 `auto_id` 必须为 `False`，并且在插入数据时用户必须提供唯一值。

In [3]:
# 示例：定义一个 INT64 类型的主键字段，并启用自动 ID 生成
pk_field_auto_id = FieldSchema(
  name="doc_id",          # 字段名
  dtype=DataType.INT64,   # 数据类型
  is_primary=True,        # 设为主键
  auto_id=True,           # 启用自动 ID 生成
  description="Document ID (auto-generated)" # 字段描述
)
print(f"主键字段 (自动ID): {pk_field_auto_id}")

主键字段 (自动ID): {'name': 'doc_id', 'description': 'Document ID (auto-generated)', 'type': <DataType.INT64: 5>, 'is_primary': True, 'auto_id': True}


In [4]:
# 示例：定义一个 VARCHAR 类型的主键字段 (需要用户提供ID)
pk_field_user_id = FieldSchema(
  name="user_uuid",
  dtype=DataType.VARCHAR,
  max_length=36,          # VARCHAR 必须指定最大长度
  is_primary=True,
  auto_id=False,          # VARCHAR 主键不能自动生成ID
  description="User unique identifier (provided by user)"
)
print(f"主键字段 (用户提供ID): {pk_field_user_id}")

主键字段 (用户提供ID): {'name': 'user_uuid', 'description': 'User unique identifier (provided by user)', 'type': <DataType.VARCHAR: 21>, 'params': {'max_length': 36}, 'is_primary': True, 'auto_id': False}


#### 如何定义自动ID (Auto ID)

如上所述，当主键字段的数据类型为 `INT64` 时，可以通过设置 `auto_id=True` 来让 Milvus 自动为插入的每条实体生成唯一的 ID。这简化了数据插入过程，因为用户无需自己管理和提供主键值。

如果 `auto_id=False` (默认值，除非 `is_primary=True` 且 `dtype=DataType.INT64` 时，`auto_id` 的行为可能因版本而异，推荐显式设置)，或者主键类型不是 `INT64`，则用户在插入数据时必须为该主键字段提供唯一值。

#### 如何定义 Vector Field (维度 Dimension)
向量字段是 Milvus 的核心，用于存储向量嵌入。
- `dtype` 必须是 `DataType.FLOAT_VECTOR` 或 `DataType.BINARY_VECTOR`。
- `dim` 参数必须指定，代表向量的维度。例如，一个由 BERT 模型生成的向量可能有 768 维。

In [5]:
# 示例：定义一个 128 维的浮点型向量字段
vector_field_128d = FieldSchema(
  name="embedding",
  dtype=DataType.FLOAT_VECTOR,
  dim=128,                 # 向量维度
  description="128-dimensional float vector embedding"
)
print(f"向量字段: {vector_field_128d}")

向量字段: {'name': 'embedding', 'description': '128-dimensional float vector embedding', 'type': <DataType.FLOAT_VECTOR: 101>, 'params': {'dim': 128}}


### 实操：创建、删除、查看 Collection

现在我们将结合以上概念，实际操作 Collection。

**1. 定义完整的 Collection Schema**

In [6]:
COLLECTION_NAME_DEMO = "my_collection" 

# 先尝试删除，确保环境干净 (如果不存在会忽略)
if client.has_collection(collection_name=COLLECTION_NAME_DEMO):
    print(f"发现已存在的 Collection '{COLLECTION_NAME_DEMO}', 将其删除。")
    client.drop_collection(collection_name=COLLECTION_NAME_DEMO)

# 定义字段
field1 = FieldSchema(name="id", dtype=DataType.INT64, is_primary=True, auto_id=True)
field2 = FieldSchema(name="title", dtype=DataType.VARCHAR, max_length=512)
field3 = FieldSchema(name="views", dtype=DataType.INT32)
field4 = FieldSchema(name="embedding", dtype=DataType.FLOAT_VECTOR, dim=8) # 使用一个较小的维度方便演示

# 定义 Schema
schema = CollectionSchema(
  fields=[field1, field2, field3, field4],
  description="A simple collection for demonstration purposes (using MilvusClient)",
  enable_dynamic_field=False # 是否启用动态字段 (Dynamic Schema)
  # True: 允许插入 Schema 中未定义的字段，这些字段将作为 JSON 存储。
  # False: (默认) 严格按照 Schema 定义插入数据。
)

print("Schema 定义完成:")

Schema 定义完成:


**2. 创建 Collection**

使用定义好的 Schema 和 Collection 名称来创建 Collection。

`consistency_level` 参数指定了 Collection 的一致性级别，影响数据可见性和搜索准确性。常用级别：
- `Strong`: 强一致性，写入操作完成后，后续读取/搜索操作能立即看到最新数据。
- `Bounded`: 有限期过时，允许在一定时间窗口内读取到稍微过时的数据，性能较好。
- `Session`: 会话一致性，同一会话内的操作保证一致性。
- `Eventually`: 最终一致性，写入操作最终会可见，延迟可能较大，性能最好。

对于大多数应用，`Bounded` 或 `Strong` 是常见的选择。默认为 `Bounded`。

In [7]:
# 创建 Collection
try:
    # 注意：consistency_level 是一个重要的参数
    client.create_collection(
        collection_name=COLLECTION_NAME_DEMO,
        schema=schema, # Pass the CollectionSchema object
        # alternatively, you can pass dim directly for simple cases:
        # collection_name=COLLECTION_NAME_DEMO,
        # dimension=8, # Only if you have one vector field and a primary key
        consistency_level="Strong" # 或者 "Bounded", "Session", "Eventually"
    )
    print(f"Collection '{COLLECTION_NAME_DEMO}' 创建成功!")
except Exception as e:
    print(f"创建 Collection '{COLLECTION_NAME_DEMO}' 失败: {e}")

Collection 'my_collection' 创建成功!


**3. 查看 Collection 信息**

我们可以查看当前 Milvus 实例中的所有 Collections，或者获取特定 Collection 的详细信息。

In [8]:
# 列出所有 Collections
all_collections = client.list_collections()
print(f"当前 Milvus 中的所有 Collections: {all_collections}")

# 检查特定 Collection 是否存在
exists = client.has_collection(collection_name=COLLECTION_NAME_DEMO)
print(f"Collection '{COLLECTION_NAME_DEMO}' 是否存在: {exists}")

if exists:
    # 获取 Collection 描述信息 (包括 schema, num_entities 等)
    desc = client.describe_collection(collection_name=COLLECTION_NAME_DEMO)
    print(f"\nCollection '{desc['collection_name']}' 描述信息:")
    print(f"  - Description: {desc['description']}")
    print(f"  - Auto ID: {desc['auto_id']}")
    print(f"  - Consistency Level: {desc['consistency_level']}")
    print(f"  - Number of Partitions: {desc['num_partitions']}") # MilvusClient returns num_entities directly in describe

    print(f"\n  - Schema Fields:")
    for field_info in desc['fields']:
        print(f"    - Name: {field_info['name']}, Type: {field_info['type']}, Is Primary: {field_info.get('is_primary', False)}"
              f", Dim: {field_info.get('params', {}).get('dim', 'N/A')}") # field_info['params']['dim'] if vector

    # 获取 Collection 统计信息 (更侧重于 row_count, sealed/growing segments等)
    stats = client.get_collection_stats(collection_name=COLLECTION_NAME_DEMO)
    print(f"\nCollection '{COLLECTION_NAME_DEMO}' 统计信息: {stats}")
    

当前 Milvus 中的所有 Collections: ['my_collection']
Collection 'my_collection' 是否存在: True

Collection 'my_collection' 描述信息:
  - Description: A simple collection for demonstration purposes (using MilvusClient)
  - Auto ID: True
  - Consistency Level: 0
  - Number of Partitions: 1

  - Schema Fields:
    - Name: id, Type: 5, Is Primary: True, Dim: N/A
    - Name: title, Type: 21, Is Primary: False, Dim: N/A
    - Name: views, Type: 4, Is Primary: False, Dim: N/A
    - Name: embedding, Type: 101, Is Primary: False, Dim: 8

Collection 'my_collection' 统计信息: {'row_count': 0}


**4. 删除 Collection**

如果不再需要某个 Collection，可以将其删除。**注意：删除操作是不可逆的，会永久移除 Collection 及其所有数据。**

In [9]:
# 删除 Collection
try:
    client.drop_collection(collection_name=COLLECTION_NAME_DEMO)
    print(f"Collection '{COLLECTION_NAME_DEMO}' 已成功删除。")
except Exception as e:
    print(f"删除 Collection '{COLLECTION_NAME_DEMO}' 失败: {e}")

# 再次检查 Collection 是否存在
exists_after_drop = client.has_collection(collection_name=COLLECTION_NAME_DEMO)
print(f"Collection '{COLLECTION_NAME_DEMO}' 是否存在 (删除后): {exists_after_drop}")


Collection 'my_collection' 已成功删除。
Collection 'my_collection' 是否存在 (删除后): False


### 实操：加载 (Load) 和释放 (Release) Collection

在 Milvus 中，数据默认存储在磁盘（或对象存储）上。为了进行高效的搜索和查询，需要将 Collection 的数据（或其一部分，如特定的 Segment 或 Partition）加载到 Milvus QueryNode 的内存中。

- **`client.load_collection()`**: 将 Collection 数据加载到内存中，使其可供搜索/查询。
- **`client.release_collection()`**: 从内存中释放 Collection 数据，以节省内存资源。数据本身不会被删除，仍然保留在存储中。

**Load 的重要性**:

- **性能**: 对内存中的数据进行搜索远快于对磁盘数据进行搜索。
- **资源管理**: 对于非常大的数据集，可能无法将所有数据同时加载到内存。Milvus 允许加载特定的分区 (Partitions) 或让 Milvus 根据资源情况自动管理。
- **准备就绪**: 只有加载完成的 Collection 才能被有效查询。

我们重新创建之前的 `simple_collection` 来演示加载和释放。

In [10]:
# 1. 重新创建 Collection (如果不存在)
if not client.has_collection(collection_name=COLLECTION_NAME_DEMO):
    # schema 变量在前面已经定义过了，这里可以直接使用
    client.create_collection(
        collection_name=COLLECTION_NAME_DEMO,
        schema=schema,
        consistency_level="Strong"
    )
    print(f"Collection '{COLLECTION_NAME_DEMO}' 已重新创建。")
else:
    print(f"Collection '{COLLECTION_NAME_DEMO}' 已存在，直接使用。")

Collection 'my_collection' 已重新创建。


In [11]:
# 2. 加载 Collection
# 在加载之前，需要先为向量字段创建索引 (后续详细介绍索引相关内容)
index_params = MilvusClient.prepare_index_params()

index_params.add_index(
    field_name="embedding",
    metric_type="COSINE",
    index_type="IVF_FLAT",
    index_name="vector_index",
    params={ "nlist": 128 }
)

client.create_index(
    collection_name=COLLECTION_NAME_DEMO,
    index_params=index_params,
    sync=False # Whether to wait for index creation to complete before returning. Defaults to True.
)

try:
    print(f"开始加载 Collection '{COLLECTION_NAME_DEMO}'...")
    client.load_collection(collection_name=COLLECTION_NAME_DEMO)
    print(f"Collection '{COLLECTION_NAME_DEMO}' 加载指令已发送。")

    # 查看加载状态 (MilvusClient 2.3+)
    # `get_load_state` returns a dictionary like {'state': <LoadState: Loaded: 1 NotLoad: 2 Loading: 3>}
    load_state_info = client.get_load_state(collection_name=COLLECTION_NAME_DEMO)
    print(f"加载状态信息: {load_state_info}")
except Exception as e:
    print(f"加载 Collection '{COLLECTION_NAME_DEMO}' 失败: {e}")
    raise


开始加载 Collection 'my_collection'...
Collection 'my_collection' 加载指令已发送。
加载状态信息: {'state': <LoadState: Loaded>}


**注意**:
- 对于一个新创建且没有数据的 Collection，`load_collection()` 操作会很快完成。
- 当 Collection 中有数据时，加载过程可能需要一些时间。
- `load_collection()` 是一个异步操作，它会立即返回。可以使用 `client.get_load_state()` 检查加载状态。
- 在对 Collection 执行搜索操作之前，必须确保它已加载。

In [12]:
# 3. 释放 Collection
try:
    print(f"开始释放 Collection '{COLLECTION_NAME_DEMO}'...")
    client.release_collection(collection_name=COLLECTION_NAME_DEMO)
    print(f"Collection '{COLLECTION_NAME_DEMO}' 已从内存中释放。")
except Exception as e:
    print(f"释放 Collection '{COLLECTION_NAME_DEMO}' 失败: {e}")

开始释放 Collection 'my_collection'...
Collection 'my_collection' 已从内存中释放。


### 实操：创建带有 TTL (Time-To-Live) 的 Collection

Time-To-Live (TTL) 是 Milvus Collection 的一个属性，它允许您为数据设置一个“存活时间”。当数据超过这个设定的时间后，Milvus 会自动将其删除。TTL 的单位是**秒**。

TTL相关操作

- 创建collection
  ```py
  client.create_collection(
        properties={ "collection.ttl.seconds": TTL_SECONDS}
    )
  ```
- 为既有collection添加TTL
  ```py
  client.alter_collection_properties(
        properties={"collection.ttl.seconds": TTL_SECONDS}
  )
  ```
- 删除 TTL
  ```py
  client.drop_collection_properties(
    property_keys=["collection.ttl.seconds"]
  )
  ```

In [13]:
COLLECTION_NAME_TTL = "my_ttl_collection"
TTL_SECONDS = 300 # 数据存活 300 秒 (5 分钟)

# 先确保 Collection 不存在
if client.has_collection(collection_name=COLLECTION_NAME_TTL):
    print(f"发现已存在的 Collection '{COLLECTION_NAME_TTL}', 将其删除。")
    client.drop_collection(collection_name=COLLECTION_NAME_TTL)

# 1. 定义字段
ttl_pk_field = FieldSchema(name="id", dtype=DataType.INT64, is_primary=True, auto_id=True)
ttl_data_field = FieldSchema(name="message", dtype=DataType.VARCHAR, max_length=1024)
ttl_vector_field = FieldSchema(name="vector", dtype=DataType.FLOAT_VECTOR, dim=4) 

# 2. 定义Schema
ttl_schema = CollectionSchema(
    fields=[ttl_pk_field, ttl_data_field, ttl_vector_field],
    description=f"Collection with TTL of {TTL_SECONDS} seconds",
    enable_dynamic_field=False
)

# 3. 创建带有 TTL 的 Collection
try:
    client.create_collection(
        collection_name=COLLECTION_NAME_TTL,
        schema=ttl_schema,
        consistency_level="Strong",
        properties={
            "collection.ttl.seconds": TTL_SECONDS
        }
    )
    print(f"带有 TTL 的 Collection '{COLLECTION_NAME_TTL}' 创建成功!")
except Exception as e:
    print(f"创建带有 TTL 的 Collection '{COLLECTION_NAME_TTL}' 失败: {e}")
    raise

# 4. 验证 TTL 设置
# Milvus 会将 TTL 信息存储在 Collection 的属性中
if client.has_collection(collection_name=COLLECTION_NAME_TTL):
    desc_ttl_collection = client.describe_collection(collection_name=COLLECTION_NAME_TTL)
    print(f"\nCollection '{COLLECTION_NAME_TTL}' 的描述信息:")
    # print(desc_ttl_collection) # 打印完整描述信息以查看结构
    
    # TTL 信息通常在 'properties' 字段下，键为 'collection.ttl.seconds'
    collection_properties = desc_ttl_collection.get('properties', {})
    actual_ttl = collection_properties.get('collection.ttl.seconds', '未设置或获取失败')
    
    print(f"  - 描述: {desc_ttl_collection.get('description')}")
    print(f"  - 配置的 TTL (秒): {actual_ttl}") # 注意：这里返回的是字符串
    
    if str(actual_ttl) == str(TTL_SECONDS):
        print(f"  - TTL 验证成功! 设置为 {TTL_SECONDS} 秒。")
    else:
        print(f"  - TTL 验证失败或不匹配。期望值: {TTL_SECONDS}, 实际值: {actual_ttl}")

    print(f"\n注意: 插入到 '{COLLECTION_NAME_TTL}' 的数据将在 {TTL_SECONDS} 秒后自动被 Milvus 清理。")
    print("要验证 TTL 效果，您可以：")
    print("  1. 向此 Collection 插入一些数据。")
    print(f"  2. 等待超过 {TTL_SECONDS} 秒。")
    print("  3. 查询 Collection，数据应该已被删除 (num_entities 变为 0 或减少)。")

# # 删除 TTL
# client.drop_collection_properties(
#     collection_name=COLLECTION_NAME_TTL,
#     property_keys=["collection.ttl.seconds"]
# )

带有 TTL 的 Collection 'my_ttl_collection' 创建成功!

Collection 'my_ttl_collection' 的描述信息:
  - 描述: Collection with TTL of 300 seconds
  - 配置的 TTL (秒): 300
  - TTL 验证成功! 设置为 300 秒。

注意: 插入到 'my_ttl_collection' 的数据将在 300 秒后自动被 Milvus 清理。
要验证 TTL 效果，您可以：
  1. 向此 Collection 插入一些数据。
  2. 等待超过 300 秒。
  3. 查询 Collection，数据应该已被删除 (num_entities 变为 0 或减少)。


### Hands-on Exercise 1: 创建一个简单的 Collection

**任务**: 创建一个名为 `book_search_mc` 的 Collection，用于存储书籍信息。该 Collection 应包含以下字段：

1.  `book_id`:
    *   类型: `INT64`
    *   属性: 主键 (Primary Key), 自动生成ID (Auto ID)
    *   描述: "Book's unique identifier"
2.  `book_title`:
    *   类型: `VARCHAR`
    *   属性: 最大长度 (max_length) 512
    *   描述: "Title of the book"
3.  `publication_year`:
    *   类型: `INT32`
    *   描述: "Year the book was published"
4.  `book_embedding`:
    *   类型: `FLOAT_VECTOR`
    *   属性: 维度 (Dimension) 768 (例如，一个常见的句子嵌入维度)
    *   描述: "Vector embedding of the book's content or title"

**步骤**:
1. 定义每个字段的 `FieldSchema`。
2. 使用这些字段定义 `CollectionSchema`。
3. 使用 `client.create_collection()` 创建 Collection。
4. 使用 `client.describe_collection()` 验证 Collection 是否成功创建，并打印其 Schema 和实体数量。
5. （可选）使用 `client.drop_collection()` 删除创建的 Collection 以清理环境。
